In [1]:
import os
import numpy as np
# import matplotlib.pyplot as plt
# import cv2
from numpy.linalg import norm
import pickle
from tqdm import tqdm, tqdm_notebook
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import matplotlib.image as mpimg
import time
from itertools import combinations 
import threading

In [2]:
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
# from tensorflow.keras.applications.resnet import ResNet101

In [3]:
model = ResNet50(weights='imagenet')
# model1 = ResNet101(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)

In [4]:
model.layers.pop()
model = Model(inputs=model.inputs, outputs=model.layers[-1].output)

In [5]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [6]:
from tensorflow.keras.preprocessing import image

Extract features from image.

In [7]:
def extract_features(img_path, model):
    img = image.load_img(img_path, target_size=(224, 224))
#     img = image.load_img(img_path, target_size=(96, 112))
    img_array = image.img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0)
    preprocessed_img = preprocess_input(expanded_img_array)
    features = model.predict(preprocessed_img)
    flattened_features = features.flatten()
    # print(flattened_features.shape)
    print("lalit   ")
    normalized_features = flattened_features / norm(flattened_features)
    return normalized_features 

Get all the extensions of all the filenames in the dataset.

In [8]:
extensions = ['.jpg', '.JPG', '.jpeg', '.JPEG', '.png', '.PNG']
def get_file_list(root_dir):
    file_list = []
    counter = 1
    for root, directories, filenames in os.walk(root_dir):
        for filename in filenames:
            if any(ext in filename for ext in extensions):
                file_list.append(os.path.join(root, filename))
                counter += 1
    return file_list

path to the datasets

In [9]:
root_dir = '/home/user/Documents/Db-Stroke-full/'

etrieving the filenames from this order

In [10]:
filenames = sorted(get_file_list(root_dir))
for i in range(len(filenames)):
    print(filenames[i],i)

/home/user/Documents/Db-Stroke-full/Hem_Complete/rgb-Case_H11_IM-0001-0009.jpg 0
/home/user/Documents/Db-Stroke-full/Hem_Complete/rgb-Case_H11_IM-0001-0010.jpg 1
/home/user/Documents/Db-Stroke-full/Hem_Complete/rgb-Case_H11_IM-0001-0011.jpg 2
/home/user/Documents/Db-Stroke-full/Hem_Complete/rgb-Case_H11_IM-0001-0012.jpg 3
/home/user/Documents/Db-Stroke-full/Hem_Complete/rgb-Case_H11_IM-0001-0013.jpg 4
/home/user/Documents/Db-Stroke-full/Hem_Complete/rgb-Case_H11_IM-0001-0014.jpg 5
/home/user/Documents/Db-Stroke-full/Hem_Complete/rgb-Case_H11_IM-0001-0015.jpg 6
/home/user/Documents/Db-Stroke-full/Hem_Complete/rgb-Case_H11_IM-0001-0016.jpg 7
/home/user/Documents/Db-Stroke-full/Hem_Complete/rgb-Case_H11_IM-0001-0017.jpg 8
/home/user/Documents/Db-Stroke-full/Hem_Complete/rgb-Case_H11_IM-0001-0018.jpg 9
/home/user/Documents/Db-Stroke-full/Hem_Complete/rgb-Case_H11_IM-0001-0021.jpg 10
/home/user/Documents/Db-Stroke-full/Hem_Complete/rgb-Case_H11_IM-0001-0022.jpg 11
/home/user/Documents/Db-St

/home/user/Documents/Db-Stroke-full/Hem_Complete/rgb-Case_H35_IM-0001-0050.jpg 381
/home/user/Documents/Db-Stroke-full/Hem_Complete/rgb-Case_H35_IM-0001-0051.jpg 382
/home/user/Documents/Db-Stroke-full/Hem_Complete/rgb-Case_H35_IM-0001-0052.jpg 383
/home/user/Documents/Db-Stroke-full/Hem_Complete/rgb-Case_H35_IM-0001-0053.jpg 384
/home/user/Documents/Db-Stroke-full/Hem_Complete/rgb-Case_H35_IM-0001-0054.jpg 385
/home/user/Documents/Db-Stroke-full/Hem_Complete/rgb-Case_H35_IM-0001-0055.jpg 386
/home/user/Documents/Db-Stroke-full/Hem_Complete/rgb-Case_H35_IM-0001-0056.jpg 387
/home/user/Documents/Db-Stroke-full/Hem_Complete/rgb-Case_H36_IM-0001-0030.jpg 388
/home/user/Documents/Db-Stroke-full/Hem_Complete/rgb-Case_H36_IM-0001-0031.jpg 389
/home/user/Documents/Db-Stroke-full/Hem_Complete/rgb-Case_H36_IM-0001-0032.jpg 390
/home/user/Documents/Db-Stroke-full/Hem_Complete/rgb-Case_H36_IM-0001-0033.jpg 391
/home/user/Documents/Db-Stroke-full/Hem_Complete/rgb-Case_H36_IM-0001-0034.jpg 392
/hom

/home/user/Documents/Db-Stroke-full/Infarct_Complete/rgb-infarct-im_Case113_IM-0001-0030.jpg 1280
/home/user/Documents/Db-Stroke-full/Infarct_Complete/rgb-infarct-im_Case113_IM-0001-0031.jpg 1281
/home/user/Documents/Db-Stroke-full/Infarct_Complete/rgb-infarct-im_Case113_IM-0001-0032.jpg 1282
/home/user/Documents/Db-Stroke-full/Infarct_Complete/rgb-infarct-im_Case113_IM-0001-0033.jpg 1283
/home/user/Documents/Db-Stroke-full/Infarct_Complete/rgb-infarct-im_Case113_IM-0001-0034.jpg 1284
/home/user/Documents/Db-Stroke-full/Infarct_Complete/rgb-infarct-im_Case113_IM-0001-0035.jpg 1285
/home/user/Documents/Db-Stroke-full/Infarct_Complete/rgb-infarct-im_Case113_IM-0001-0036.jpg 1286
/home/user/Documents/Db-Stroke-full/Infarct_Complete/rgb-infarct-im_Case113_IM-0001-0037.jpg 1287
/home/user/Documents/Db-Stroke-full/Infarct_Complete/rgb-infarct-im_Case113_IM-0001-0038.jpg 1288
/home/user/Documents/Db-Stroke-full/Infarct_Complete/rgb-infarct-im_Case113_IM-0001-0039.jpg 1289
/home/user/Documents

/home/user/Documents/Db-Stroke-full/Infarct_Complete/rgb-infarct-im_Case196_IM-0001-0036.jpg 1904
/home/user/Documents/Db-Stroke-full/Infarct_Complete/rgb-infarct-im_Case196_IM-0001-0037.jpg 1905
/home/user/Documents/Db-Stroke-full/Infarct_Complete/rgb-infarct-im_Case196_IM-0001-0038.jpg 1906
/home/user/Documents/Db-Stroke-full/Infarct_Complete/rgb-infarct-im_Case196_IM-0001-0039.jpg 1907
/home/user/Documents/Db-Stroke-full/Infarct_Complete/rgb-infarct-im_Case196_IM-0001-0040.jpg 1908
/home/user/Documents/Db-Stroke-full/Infarct_Complete/rgb-infarct-im_Case196_IM-0001-0041.jpg 1909
/home/user/Documents/Db-Stroke-full/Infarct_Complete/rgb-infarct-im_Case196_IM-0001-0042.jpg 1910
/home/user/Documents/Db-Stroke-full/Infarct_Complete/rgb-infarct-im_Case196_IM-0001-0043.jpg 1911
/home/user/Documents/Db-Stroke-full/Infarct_Complete/rgb-infarct-im_Case196_IM-0001-0044.jpg 1912
/home/user/Documents/Db-Stroke-full/Infarct_Complete/rgb-infarct-im_Case196_IM-0001-0045.jpg 1913
/home/user/Documents

/home/user/Documents/Db-Stroke-full/Norm_Complete/rgb-normal-im_Case119_IM-0001-0026.jpg 2991
/home/user/Documents/Db-Stroke-full/Norm_Complete/rgb-normal-im_Case119_IM-0001-0027.jpg 2992
/home/user/Documents/Db-Stroke-full/Norm_Complete/rgb-normal-im_Case119_IM-0001-0028.jpg 2993
/home/user/Documents/Db-Stroke-full/Norm_Complete/rgb-normal-im_Case119_IM-0001-0029.jpg 2994
/home/user/Documents/Db-Stroke-full/Norm_Complete/rgb-normal-im_Case119_IM-0001-0030.jpg 2995
/home/user/Documents/Db-Stroke-full/Norm_Complete/rgb-normal-im_Case119_IM-0001-0031.jpg 2996
/home/user/Documents/Db-Stroke-full/Norm_Complete/rgb-normal-im_Case119_IM-0001-0032.jpg 2997
/home/user/Documents/Db-Stroke-full/Norm_Complete/rgb-normal-im_Case119_IM-0001-0033.jpg 2998
/home/user/Documents/Db-Stroke-full/Norm_Complete/rgb-normal-im_Case119_IM-0001-0034.jpg 2999
/home/user/Documents/Db-Stroke-full/Norm_Complete/rgb-normal-im_Case119_IM-0001-0035.jpg 3000
/home/user/Documents/Db-Stroke-full/Norm_Complete/rgb-normal

...

/home/user/Documents/Db-Stroke-full/Norm_Complete/rgb-normal-im_Case171_IM-0001-0017.jpg 4770
/home/user/Documents/Db-Stroke-full/Norm_Complete/rgb-normal-im_Case171_IM-0001-0018.jpg 4771
/home/user/Documents/Db-Stroke-full/Norm_Complete/rgb-normal-im_Case171_IM-0001-0019.jpg 4772
/home/user/Documents/Db-Stroke-full/Norm_Complete/rgb-normal-im_Case171_IM-0001-0020.jpg 4773
/home/user/Documents/Db-Stroke-full/Norm_Complete/rgb-normal-im_Case171_IM-0001-0021.jpg 4774
/home/user/Documents/Db-Stroke-full/Norm_Complete/rgb-normal-im_Case171_IM-0001-0022.jpg 4775
/home/user/Documents/Db-Stroke-full/Norm_Complete/rgb-normal-im_Case171_IM-0001-0023.jpg 4776
/home/user/Documents/Db-Stroke-full/Norm_Complete/rgb-normal-im_Case171_IM-0001-0024.jpg 4777
/home/user/Documents/Db-Stroke-full/Norm_Complete/rgb-normal-im_Case171_IM-0001-0025.jpg 4778
/home/user/Documents/Db-Stroke-full/Norm_Complete/rgb-normal-im_Case171_IM-0001-0026.jpg 4779
/home/user/Documents/Db-Stroke-full/Norm_Complete/rgb-normal

/home/user/Documents/Db-Stroke-full/Norm_Complete/rgb-normal-im_Case1_IM-0001-0032.jpg 5621
/home/user/Documents/Db-Stroke-full/Norm_Complete/rgb-normal-im_Case1_IM-0001-0033.jpg 5622
/home/user/Documents/Db-Stroke-full/Norm_Complete/rgb-normal-im_Case1_IM-0001-0034.jpg 5623
/home/user/Documents/Db-Stroke-full/Norm_Complete/rgb-normal-im_Case1_IM-0001-0035.jpg 5624
/home/user/Documents/Db-Stroke-full/Norm_Complete/rgb-normal-im_Case1_IM-0001-0036.jpg 5625
/home/user/Documents/Db-Stroke-full/Norm_Complete/rgb-normal-im_Case1_IM-0001-0037.jpg 5626
/home/user/Documents/Db-Stroke-full/Norm_Complete/rgb-normal-im_Case1_IM-0001-0038.jpg 5627
/home/user/Documents/Db-Stroke-full/Norm_Complete/rgb-normal-im_Case1_IM-0001-0039.jpg 5628
/home/user/Documents/Db-Stroke-full/Norm_Complete/rgb-normal-im_Case1_IM-0001-0040.jpg 5629
/home/user/Documents/Db-Stroke-full/Norm_Complete/rgb-normal-im_Case1_IM-0001-0041.jpg 5630
/home/user/Documents/Db-Stroke-full/Norm_Complete/rgb-normal-im_Case1_IM-0001-00

/home/user/Documents/Db-Stroke-full/Norm_Complete/rgb-normal-im_Case39_IM-0001-0036.jpg 6241
/home/user/Documents/Db-Stroke-full/Norm_Complete/rgb-normal-im_Case39_IM-0001-0037.jpg 6242
/home/user/Documents/Db-Stroke-full/Norm_Complete/rgb-normal-im_Case39_IM-0001-0038.jpg 6243
/home/user/Documents/Db-Stroke-full/Norm_Complete/rgb-normal-im_Case39_IM-0001-0039.jpg 6244
/home/user/Documents/Db-Stroke-full/Norm_Complete/rgb-normal-im_Case39_IM-0001-0040.jpg 6245
/home/user/Documents/Db-Stroke-full/Norm_Complete/rgb-normal-im_Case39_IM-0001-0041.jpg 6246
/home/user/Documents/Db-Stroke-full/Norm_Complete/rgb-normal-im_Case39_IM-0001-0042.jpg 6247
/home/user/Documents/Db-Stroke-full/Norm_Complete/rgb-normal-im_Case39_IM-0001-0043.jpg 6248
/home/user/Documents/Db-Stroke-full/Norm_Complete/rgb-normal-im_Case39_IM-0001-0044.jpg 6249
/home/user/Documents/Db-Stroke-full/Norm_Complete/rgb-normal-im_Case39_IM-0001-0045.jpg 6250
/home/user/Documents/Db-Stroke-full/Norm_Complete/rgb-normal-im_Case39

/home/user/Documents/Db-Stroke-full/Norm_Complete/rgb-normal-im_Case70_IM-0001-0034.jpg 7261
/home/user/Documents/Db-Stroke-full/Norm_Complete/rgb-normal-im_Case70_IM-0001-0035.jpg 7262
/home/user/Documents/Db-Stroke-full/Norm_Complete/rgb-normal-im_Case70_IM-0001-0036.jpg 7263
/home/user/Documents/Db-Stroke-full/Norm_Complete/rgb-normal-im_Case70_IM-0001-0037.jpg 7264
/home/user/Documents/Db-Stroke-full/Norm_Complete/rgb-normal-im_Case70_IM-0001-0038.jpg 7265
/home/user/Documents/Db-Stroke-full/Norm_Complete/rgb-normal-im_Case70_IM-0001-0039.jpg 7266
/home/user/Documents/Db-Stroke-full/Norm_Complete/rgb-normal-im_Case70_IM-0001-0040.jpg 7267
/home/user/Documents/Db-Stroke-full/Norm_Complete/rgb-normal-im_Case70_IM-0001-0041.jpg 7268
/home/user/Documents/Db-Stroke-full/Norm_Complete/rgb-normal-im_Case70_IM-0001-0042.jpg 7269
/home/user/Documents/Db-Stroke-full/Norm_Complete/rgb-normal-im_Case70_IM-0001-0043.jpg 7270
/home/user/Documents/Db-Stroke-full/Norm_Complete/rgb-normal-im_Case70

/home/user/Documents/Db-Stroke-full/Norm_Complete/rgb-normal-im_Case96_IM-0001-0007.jpg 8206
/home/user/Documents/Db-Stroke-full/Norm_Complete/rgb-normal-im_Case96_IM-0001-0008.jpg 8207
/home/user/Documents/Db-Stroke-full/Norm_Complete/rgb-normal-im_Case96_IM-0001-0009.jpg 8208
/home/user/Documents/Db-Stroke-full/Norm_Complete/rgb-normal-im_Case96_IM-0001-0010.jpg 8209
/home/user/Documents/Db-Stroke-full/Norm_Complete/rgb-normal-im_Case96_IM-0001-0011.jpg 8210
/home/user/Documents/Db-Stroke-full/Norm_Complete/rgb-normal-im_Case96_IM-0001-0012.jpg 8211
/home/user/Documents/Db-Stroke-full/Norm_Complete/rgb-normal-im_Case96_IM-0001-0013.jpg 8212
/home/user/Documents/Db-Stroke-full/Norm_Complete/rgb-normal-im_Case96_IM-0001-0014.jpg 8213
/home/user/Documents/Db-Stroke-full/Norm_Complete/rgb-normal-im_Case96_IM-0001-0015.jpg 8214
/home/user/Documents/Db-Stroke-full/Norm_Complete/rgb-normal-im_Case96_IM-0001-0016.jpg 8215
/home/user/Documents/Db-Stroke-full/Norm_Complete/rgb-normal-im_Case96


 Get all the features in the database for all filenames.
<br>
feature_list = []<br>
var = 1<br>
for i in tqdm_notebook(range(len(filenames))):<br>
    feature_list.append(extract_features(filenames[i], model))<br>
class BinaryStep:<br>
    def __init__(self, x):<br>
        self.x = x<br>
    def forward(self):<br>
        self.x[self.x <= 0.00] = 0<br>
        self.x[self.x > 0.00] = 1<br>
        return self.x<br>
    def backward(self):<br>
        return self.x<br>
    <br>
Load pca object. 


In [11]:
feature_list = []
var = 1
for i in tqdm_notebook(range(len(filenames))):
    feature_list.append(extract_features(filenames[i], model))


/home/user/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
l

lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
l

lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
l

lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
l

...

lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
l

lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
l

lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
l

lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
l

lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
lalit   
l

In [12]:
class BinaryStep:
    def __init__(self, x):
        self.x = x

    def forward(self):
        self.x[self.x <= 0.00] = 0
        self.x[self.x > 0.00] = 1
        return self.x

    def backward(self):
        return self.x

In [13]:
pca = pickle.load(open('./pca_128.pickle', 'rb'))
print(pca)

PCA(copy=True, iterated_power='auto', n_components=128, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)


/home/user/anaconda3/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator PCA from version 0.22.2.post1 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [14]:
features_to_binary = BinaryStep(pca.transform(feature_list))
features_forward = features_to_binary.forward()
print(features_forward.shape)

(8379, 128)


In [15]:
from sklearn.decomposition import PCA
num_feature_dimensions=128
pca = PCA(n_components = num_feature_dimensions)
pca.fit(feature_list)

features_to_binary = BinaryStep(pca.transform(feature_list))
features_forward = features_to_binary.forward()


applying PCA  and converting to binary 
<br>
features_to_binary = BinaryStep(pca.transform(feature_list))<br>
features_forward = features_to_binary.forward()<br>
print(features_forward.shape)<br>
Execute this only when new PCA needs to be computed (when a large number of images are added to database)


# 
 PCA is applied to get the number of dimensions to get hashcodes.
<br>
from sklearn.decomposition import PCA<br>
num_feature_dimensions=128<br>
pca = PCA(n_components = num_feature_dimensions)<br>
pca.fit(feature_list)<br>
features_to_binary = BinaryStep(pca.transform(feature_list))<br>
features_forward = features_to_binary.forward()<br>
Hash codes. 


In [16]:
print(features_forward.shape)

(8379, 128)



 you can dump the file and use it later
<br>
onvert hash-bits into a single hash code. conversion to string 


In [17]:
def convert(list): 
    s = [str(int(i)) for i in list] 
    mystring = ""
    for digit in s:
        mystring += str(int(digit))
    return(mystring) 

In [18]:
import pymongo
from pymongo.collation import Collation
import locale

In [19]:
pip install pymongo

Note: you may need to restart the kernel to use updated packages.


In [20]:
client = pymongo.MongoClient('mongodb://127.0.0.1:27017')

In [21]:
database=client['HashCodes']            #name of database = hashcodes  

In [22]:
collection=database.object_categories             #name of collection = codes

In [23]:
def insert_record(query,address):
    data = {'hashcode':query,
            'parts0':query[0:16],
            'parts1':query[16:32],
            'parts2':query[32:48],
            'parts3':query[48:64],
            'parts4':query[64:80],
            'parts5':query[80:96],
            'parts6':query[96:112],
            'parts7':query[112:128],
            'img_address':address
            }
    collection.insert_one(data)

In [24]:
in0 = database.object_categories.create_index([('parts0',1)])
in1 = database.object_categories.create_index([('parts1',1)])
in2 = database.object_categories.create_index([('parts2',1)])
in3 = database.object_categories.create_index([('parts3',1)])
in4 = database.object_categories.create_index([('parts4',1)])
in5 = database.object_categories.create_index([('parts5',1)])
in6 = database.object_categories.create_index([('parts6',1)])
in7 = database.object_categories.create_index([('parts7',1)])
database.object_categories.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]},
 'parts0_1': {'v': 2, 'key': [('parts0', 1)]},
 'parts1_1': {'v': 2, 'key': [('parts1', 1)]},
 'parts2_1': {'v': 2, 'key': [('parts2', 1)]},
 'parts3_1': {'v': 2, 'key': [('parts3', 1)]},
 'parts4_1': {'v': 2, 'key': [('parts4', 1)]},
 'parts5_1': {'v': 2, 'key': [('parts5', 1)]},
 'parts6_1': {'v': 2, 'key': [('parts6', 1)]},
 'parts7_1': {'v': 2, 'key': [('parts7', 1)]}}

database.collection.drop_indexes()

In [25]:
features_new = []
for i in tqdm_notebook(range(len(feature_list))):
    features_new.append(convert(features_forward[i]))

/home/user/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


database.collection.find({'parts0':'0000000000000000'},{})


New features list consisting of the appended hashcodes
<br>
features_new = []<br>
for i in tqdm_notebook(range(len(feature_list))):<br>
    features_new.append(convert(features_forward[i]))<br>
calling insert funtion 


In [26]:
for i in range (0,len(filenames)):
    insert_record(features_new[i],filenames[i])
    print("inserting",i)

inserting 0
inserting 1
inserting 2
inserting 3
inserting 4
inserting 5
inserting 6
inserting 7
inserting 8
inserting 9
inserting 10
inserting 11
inserting 12
inserting 13
inserting 14
inserting 15
inserting 16
inserting 17
inserting 18
inserting 19
inserting 20
inserting 21
inserting 22
inserting 23
inserting 24
inserting 25
inserting 26
inserting 27
inserting 28
inserting 29
inserting 30
inserting 31
inserting 32
inserting 33
inserting 34
inserting 35
inserting 36
inserting 37
inserting 38
inserting 39
inserting 40
inserting 41
inserting 42
inserting 43
inserting 44
inserting 45
inserting 46
inserting 47
inserting 48
inserting 49
inserting 50
inserting 51
inserting 52
inserting 53
inserting 54
inserting 55
inserting 56
inserting 57
inserting 58
inserting 59
inserting 60
inserting 61
inserting 62
inserting 63
inserting 64
inserting 65
inserting 66
inserting 67
inserting 68
inserting 69
inserting 70
inserting 71
inserting 72
inserting 73
inserting 74
inserting 75
inserting 76
inserting

inserting 735
inserting 736
inserting 737
inserting 738
inserting 739
inserting 740
inserting 741
inserting 742
inserting 743
inserting 744
inserting 745
inserting 746
inserting 747
inserting 748
inserting 749
inserting 750
inserting 751
inserting 752
inserting 753
inserting 754
inserting 755
inserting 756
inserting 757
inserting 758
inserting 759
inserting 760
inserting 761
inserting 762
inserting 763
inserting 764
inserting 765
inserting 766
inserting 767
inserting 768
inserting 769
inserting 770
inserting 771
inserting 772
inserting 773
inserting 774
inserting 775
inserting 776
inserting 777
inserting 778
inserting 779
inserting 780
inserting 781
inserting 782
inserting 783
inserting 784
inserting 785
inserting 786
inserting 787
inserting 788
inserting 789
inserting 790
inserting 791
inserting 792
inserting 793
inserting 794
inserting 795
inserting 796
inserting 797
inserting 798
inserting 799
inserting 800
inserting 801
inserting 802
inserting 803
inserting 804
inserting 805
insert

inserting 1380
inserting 1381
inserting 1382
inserting 1383
inserting 1384
inserting 1385
inserting 1386
inserting 1387
inserting 1388
inserting 1389
inserting 1390
inserting 1391
inserting 1392
inserting 1393
inserting 1394
inserting 1395
inserting 1396
inserting 1397
inserting 1398
inserting 1399
inserting 1400
inserting 1401
inserting 1402
inserting 1403
inserting 1404
inserting 1405
inserting 1406
inserting 1407
inserting 1408
inserting 1409
inserting 1410
inserting 1411
inserting 1412
inserting 1413
inserting 1414
inserting 1415
inserting 1416
inserting 1417
inserting 1418
inserting 1419
inserting 1420
inserting 1421
inserting 1422
inserting 1423
inserting 1424
inserting 1425
inserting 1426
inserting 1427
inserting 1428
inserting 1429
inserting 1430
inserting 1431
inserting 1432
inserting 1433
inserting 1434
inserting 1435
inserting 1436
inserting 1437
inserting 1438
inserting 1439
inserting 1440
inserting 1441
inserting 1442
inserting 1443
inserting 1444
inserting 1445
inserting 

inserting 2030
inserting 2031
inserting 2032
inserting 2033
inserting 2034
inserting 2035
inserting 2036
inserting 2037
inserting 2038
inserting 2039
inserting 2040
inserting 2041
inserting 2042
inserting 2043
inserting 2044
inserting 2045
inserting 2046
inserting 2047
inserting 2048
inserting 2049
inserting 2050
inserting 2051
inserting 2052
inserting 2053
inserting 2054
inserting 2055
inserting 2056
inserting 2057
inserting 2058
inserting 2059
inserting 2060
inserting 2061
inserting 2062
inserting 2063
inserting 2064
inserting 2065
inserting 2066
inserting 2067
inserting 2068
inserting 2069
inserting 2070
inserting 2071
inserting 2072
inserting 2073
inserting 2074
inserting 2075
inserting 2076
inserting 2077
inserting 2078
inserting 2079
inserting 2080
inserting 2081
inserting 2082
inserting 2083
inserting 2084
inserting 2085
inserting 2086
inserting 2087
inserting 2088
inserting 2089
inserting 2090
inserting 2091
inserting 2092
inserting 2093
inserting 2094
inserting 2095
inserting 

inserting 2675
inserting 2676
inserting 2677
inserting 2678
inserting 2679
inserting 2680
inserting 2681
inserting 2682
inserting 2683
inserting 2684
inserting 2685
inserting 2686
inserting 2687
inserting 2688
inserting 2689
inserting 2690
inserting 2691
inserting 2692
inserting 2693
inserting 2694
inserting 2695
inserting 2696
inserting 2697
inserting 2698
inserting 2699
inserting 2700
inserting 2701
inserting 2702
inserting 2703
inserting 2704
inserting 2705
inserting 2706
inserting 2707
inserting 2708
inserting 2709
inserting 2710
inserting 2711
inserting 2712
inserting 2713
inserting 2714
inserting 2715
inserting 2716
inserting 2717
inserting 2718
inserting 2719
inserting 2720
inserting 2721
inserting 2722
inserting 2723
inserting 2724
inserting 2725
inserting 2726
inserting 2727
inserting 2728
inserting 2729
inserting 2730
inserting 2731
inserting 2732
inserting 2733
inserting 2734
inserting 2735
inserting 2736
inserting 2737
inserting 2738
inserting 2739
inserting 2740
inserting 

...

inserting 5545
inserting 5546
inserting 5547
inserting 5548
inserting 5549
inserting 5550
inserting 5551
inserting 5552
inserting 5553
inserting 5554
inserting 5555
inserting 5556
inserting 5557
inserting 5558
inserting 5559
inserting 5560
inserting 5561
inserting 5562
inserting 5563
inserting 5564
inserting 5565
inserting 5566
inserting 5567
inserting 5568
inserting 5569
inserting 5570
inserting 5571
inserting 5572
inserting 5573
inserting 5574
inserting 5575
inserting 5576
inserting 5577
inserting 5578
inserting 5579
inserting 5580
inserting 5581
inserting 5582
inserting 5583
inserting 5584
inserting 5585
inserting 5586
inserting 5587
inserting 5588
inserting 5589
inserting 5590
inserting 5591
inserting 5592
inserting 5593
inserting 5594
inserting 5595
inserting 5596
inserting 5597
inserting 5598
inserting 5599
inserting 5600
inserting 5601
inserting 5602
inserting 5603
inserting 5604
inserting 5605
inserting 5606
inserting 5607
inserting 5608
inserting 5609
inserting 5610
inserting 

inserting 6252
inserting 6253
inserting 6254
inserting 6255
inserting 6256
inserting 6257
inserting 6258
inserting 6259
inserting 6260
inserting 6261
inserting 6262
inserting 6263
inserting 6264
inserting 6265
inserting 6266
inserting 6267
inserting 6268
inserting 6269
inserting 6270
inserting 6271
inserting 6272
inserting 6273
inserting 6274
inserting 6275
inserting 6276
inserting 6277
inserting 6278
inserting 6279
inserting 6280
inserting 6281
inserting 6282
inserting 6283
inserting 6284
inserting 6285
inserting 6286
inserting 6287
inserting 6288
inserting 6289
inserting 6290
inserting 6291
inserting 6292
inserting 6293
inserting 6294
inserting 6295
inserting 6296
inserting 6297
inserting 6298
inserting 6299
inserting 6300
inserting 6301
inserting 6302
inserting 6303
inserting 6304
inserting 6305
inserting 6306
inserting 6307
inserting 6308
inserting 6309
inserting 6310
inserting 6311
inserting 6312
inserting 6313
inserting 6314
inserting 6315
inserting 6316
inserting 6317
inserting 

inserting 6851
inserting 6852
inserting 6853
inserting 6854
inserting 6855
inserting 6856
inserting 6857
inserting 6858
inserting 6859
inserting 6860
inserting 6861
inserting 6862
inserting 6863
inserting 6864
inserting 6865
inserting 6866
inserting 6867
inserting 6868
inserting 6869
inserting 6870
inserting 6871
inserting 6872
inserting 6873
inserting 6874
inserting 6875
inserting 6876
inserting 6877
inserting 6878
inserting 6879
inserting 6880
inserting 6881
inserting 6882
inserting 6883
inserting 6884
inserting 6885
inserting 6886
inserting 6887
inserting 6888
inserting 6889
inserting 6890
inserting 6891
inserting 6892
inserting 6893
inserting 6894
inserting 6895
inserting 6896
inserting 6897
inserting 6898
inserting 6899
inserting 6900
inserting 6901
inserting 6902
inserting 6903
inserting 6904
inserting 6905
inserting 6906
inserting 6907
inserting 6908
inserting 6909
inserting 6910
inserting 6911
inserting 6912
inserting 6913
inserting 6914
inserting 6915
inserting 6916
inserting 

inserting 7452
inserting 7453
inserting 7454
inserting 7455
inserting 7456
inserting 7457
inserting 7458
inserting 7459
inserting 7460
inserting 7461
inserting 7462
inserting 7463
inserting 7464
inserting 7465
inserting 7466
inserting 7467
inserting 7468
inserting 7469
inserting 7470
inserting 7471
inserting 7472
inserting 7473
inserting 7474
inserting 7475
inserting 7476
inserting 7477
inserting 7478
inserting 7479
inserting 7480
inserting 7481
inserting 7482
inserting 7483
inserting 7484
inserting 7485
inserting 7486
inserting 7487
inserting 7488
inserting 7489
inserting 7490
inserting 7491
inserting 7492
inserting 7493
inserting 7494
inserting 7495
inserting 7496
inserting 7497
inserting 7498
inserting 7499
inserting 7500
inserting 7501
inserting 7502
inserting 7503
inserting 7504
inserting 7505
inserting 7506
inserting 7507
inserting 7508
inserting 7509
inserting 7510
inserting 7511
inserting 7512
inserting 7513
inserting 7514
inserting 7515
inserting 7516
inserting 7517
inserting 

inserting 8023
inserting 8024
inserting 8025
inserting 8026
inserting 8027
inserting 8028
inserting 8029
inserting 8030
inserting 8031
inserting 8032
inserting 8033
inserting 8034
inserting 8035
inserting 8036
inserting 8037
inserting 8038
inserting 8039
inserting 8040
inserting 8041
inserting 8042
inserting 8043
inserting 8044
inserting 8045
inserting 8046
inserting 8047
inserting 8048
inserting 8049
inserting 8050
inserting 8051
inserting 8052
inserting 8053
inserting 8054
inserting 8055
inserting 8056
inserting 8057
inserting 8058
inserting 8059
inserting 8060
inserting 8061
inserting 8062
inserting 8063
inserting 8064
inserting 8065
inserting 8066
inserting 8067
inserting 8068
inserting 8069
inserting 8070
inserting 8071
inserting 8072
inserting 8073
inserting 8074
inserting 8075
inserting 8076
inserting 8077
inserting 8078
inserting 8079
inserting 8080
inserting 8081
inserting 8082
inserting 8083
inserting 8084
inserting 8085
inserting 8086
inserting 8087
inserting 8088
inserting 